In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install "unsloth[torch]" trl datasets accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 139.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393

In [1]:
sft_model_path = "/content/drive/MyDrive/tinyllama-lora-sft-tuned-model"


In [11]:
from unsloth import FastLanguageModel
from trl import DPOTrainer, DPOConfig
from datasets import load_dataset
import torch 

# Load model and tokenizer from your fine-tuned path
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=sft_model_path,
    dtype=torch.float16, 
    load_in_4bit=True  
)


==((====))==  Unsloth 2025.5.9: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
unsloth/tinyllama-chat-bnb-4bit does not have a padding token! Will use pad_token = <unk>.


In [12]:
dataset = load_dataset("yitingxie/rlhf-reward-datasets", split="train").select(range(10000))
print(dataset[0])

{'prompt': "\n\nHuman: Is it legal to have a dog that is half wolf?\n\nAssistant: It sounds like you’re asking whether it’s legal to have a dog that is partially wolf, as opposed to all wolf.  There’s a bit of ambiguity to your question.  What’s the context of your interest in the law?\n\nHuman: I'd like to get a puppy that is half wolf and half husky.\n\nAssistant: Ah, now I understand.  We can look up some details on the breed of wolf and husky dogs and see if they’re considered “hybrids”.\n\nHuman: They are called wolf dogs. It doesn't matter if you want to call them hybrids though.\n\nAssistant: That makes it sound like you’re seeking information to get an opinion on whether your dog is legal or not.\n\nHuman: Yes, does it depend on the state?\n\n", 'chosen': 'Assistant: Can I ask why you’re so interested in getting a dog that is a hybrid?', 'rejected': 'Assistant: There are a lot of state laws about “hybrid” or “mutant” dogs.  Usually a dog is defined as a certain “breed” or “type

In [13]:
training_args = DPOConfig(
    output_dir="/content/drive/MyDrive/tinyllama-dpo-trained",
    logging_steps=100,
    bf16=False, 
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1
)


In [14]:
trainer = DPOTrainer(model=model, args=training_args, processing_class=tokenizer, train_dataset=dataset)


In [15]:
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 3 | Total steps = 3,750
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 2,252,800/4,000,000,000 (0.06% trained)


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
100,1.015500,-0.973149,-0.738207,0.430000,-0.234942,-168.441177,-143.997192,-3.433339,-3.432140,0,0,0,0
200,0.829900,-0.462434,-0.388536,0.471250,-0.073898,-164.414917,-138.943909,-3.479740,-3.445623,No Log,No Log,No Log,No Log
300,0.771000,-0.254974,-0.226363,0.486250,-0.028611,-162.699997,-140.138840,-3.478656,-3.478300,No Log,No Log,No Log,No Log
400,0.712100,-0.261978,-0.333475,0.532500,0.071497,-161.803421,-142.038376,-3.527026,-3.514725,No Log,No Log,No Log,No Log
500,0.680700,-0.317424,-0.478328,0.591250,0.160904,-156.492020,-136.837540,-3.513577,-3.497554,No Log,No Log,No Log,No Log
600,0.671400,-0.421424,-0.657448,0.608750,0.236025,-169.277054,-148.219772,-3.494584,-3.476874,No Log,No Log,No Log,No Log
700,0.680400,-0.466313,-0.700195,0.605000,0.233881,-163.347778,-142.378922,-3.510435,-3.482654,No Log,No Log,No Log,No Log
800,0.680600,-0.349397,-0.549014,0.608750,0.199617,-162.626938,-134.777695,-3.524477,-3.502954,No Log,No Log,No Log,No Log
900,0.690200,-0.442022,-0.626701,0.585000,0.184678,-159.787704,-144.775162,-3.510712,-3.480891,No Log,No Log,No Log,No Log
1000,0.671300,-0.396406,-0.636699,0.616250,0.240294,-165.338989,-145.988831,-3.505806,-3.487820,No Log,No Log,No Log,No Log


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

TrainOutput(global_step=3750, training_loss=0.5622966364542643, metrics={'train_runtime': 2779.4027, 'train_samples_per_second': 10.794, 'train_steps_per_second': 1.349, 'total_flos': 0.0, 'train_loss': 0.5622966364542643, 'epoch': 3.0})

In [17]:
# Load the model with adapters (same as before)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="/content/drive/MyDrive/tinyllama-dpo-trained/checkpoint-3750",  # Use final checkpoint
    dtype=torch.float16,
    load_in_4bit=False  # Set to False before merging
)

# ✅ Merge the LoRA adapters into the base model
model = model.merge_and_unload()

# ✅ Save the merged model and tokenizer
save_path = "/content/drive/MyDrive/tinyllama-dpo-merged-final"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"✅ Final merged model saved to: {save_path}")

"""
So I have trained the SFT model with LoRA adapters
and in DPO we trained the same SFT model with reward data (still using LoRA)
then finally we merged the adapters into the base model
so the final model has:

base TinyLlama knowledge
SFT instruction knowledge
reward-based DPO tuning”
"""

==((====))==  Unsloth 2025.5.9: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Final merged model saved to: /content/drive/MyDrive/tinyllama-dpo-merged-final
